In [1]:
from typing import List, NamedTuple
import numpy as np
import jupyros as jr 
import os 
from visualization_msgs.msg import Marker, MarkerArray
from visualization_msgs.msg import Marker
import moveit_commander
from visualization_msgs.msg import Marker
import numpy as np
from scipy.spatial.transform import Rotation
from franka_msgs.msg import ErrorRecoveryAction, ErrorRecoveryActionGoal
import rospy
from gazebo_msgs.srv import (
    GetModelState,
    GetWorldProperties,
    GetModelProperties,
)
from geometry_msgs.msg import PoseStamped


Failed to import pyassimp, see https://github.com/ros-planning/moveit/issues/86 for more info


In [2]:
rospy.init_node("grasp_executor")


In [3]:
import importlib
import src.env.moveit_interface as moveit_interface
importlib.reload(moveit_interface)
import src.env.utils
importlib.reload(src.env.utils)
from src.env.utils import pose_to_list

In [4]:
import moveit_commander
moveit_commander.MoveGroupCommander("panda_manipulator").get_current_joint_values()

[-0.06461474670798939,
 -0.5838553026391198,
 -0.3817254430121313,
 -1.9759290387751651,
 -0.19241680207052791,
 1.4353618116417994,
 0.3596019714479075]

In [5]:
init_pose = [
-0.0001513652608586824,
-0.7856258935003497,
9.991577917034533e-06,
-2.3559505836972523,
2.0776787743592706e-05,
1.5717418653690114,
0.7853832208311591
]
grasp_controller = moveit_interface.GraspExecutor("world", reset_pose=init_pose)
print("Resetted")
grasp_controller.reset() 


Loading static scene information
Set up Franka API. Ready to go!
Resetted


In [6]:
grasp_controller.objects


{'ground_plane': {},
 'table': {},
 'pick_tray': {},
 'place_tray': {},
 'stone': {}}

## Grasp

In [7]:
gazebo_node_name = 'gazebo'
object_name = 'stone'
frame_id = 'world'
get_model_state = rospy.ServiceProxy(f'{gazebo_node_name}/get_model_state', GetModelState)


In [8]:
object_state = get_model_state(object_name, frame_id)
object_pose = object_state.pose
print(object_pose)
pos, rot = pose_to_list(object_pose)
# print(pos, rot)

position: 
  x: 0.0002860000530031958
  y: -0.2219720009310199
  z: 0.46316699994401617
orientation: 
  x: 1.4547191945983895e-08
  y: 8.280556300674997e-10
  z: 2.1379716566815116e-13
  w: 1.0


In [9]:
grasp_controller.grasp(np.array(pos) + np.array([0,0,0.01], dtype=float), np.array([1, 0, 0, 0], dtype=float))
# lift
grasp_controller.move_to_pose(np.array(pos) + np.array([0,0,0.1], dtype=float), np.array([1, 0, 0, 0], dtype=float)) 

Moving to pre-grasp pose
Moved to pre grasp. Remmoving object
Moving to grasp pose
Closing gripper


True

In [10]:
import time 
time.sleep(5)

## Place

In [11]:
gazebo_node_name = 'gazebo'
object_name = 'place_tray'
frame_id = 'world'
get_model_state = rospy.ServiceProxy(f'{gazebo_node_name}/get_model_state', GetModelState)


In [12]:
object_state = get_model_state(object_name, frame_id)
object_pose = object_state.pose
print(object_pose)
pos, rot = pose_to_list(object_pose)

position: 
  x: 0.004103
  y: 0.219654
  z: 0.419764
orientation: 
  x: 0.0
  y: 0.0
  z: 0.0
  w: 1.0


In [13]:
grasp_controller.place(np.array(pos) + np.array([0,0,0.1], dtype=float), np.array([1, 0, 0, 0], dtype=float))

Moving to place pose
Moved to place. Remmoving object
Opening gripper


True

## RVIZ markers

In [14]:
# jr.publish('/end_effector_pose', PoseStamped)

In [15]:
# pos = [0.15 + 0.3, 0.10, 0.6]
# ori = np.array([0, -0.923, 0.382, 0])
# ori = ori / np.linalg.norm(ori)
# print("Moving to", pos)